In [98]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import pandas as pd
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import nn
from tensorflow.python.ops import math_ops
from sklearn.preprocessing import StandardScaler

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

In [99]:
def transform_dataset_different_timesteps(dataset):
    """
    dataset: is the original dataset with all numeric columns

    This function returns a tuple of lenght 4.
    encoder_input_data: list of list
    decoder_input_data: list of FG values
    decoder_target_data: list of FG values to predict
    buckets: dictionary with info of samples with the same timesteps. Key is the number of timesteps, 
    and value is a list with the indexes of the samples which have this number of timesteps"""

    # Represents the encoder input data for each sample
    encoder_input_data = []
    # Represents the decoder input data for each sample
    decoder_input_data= []
    # Represents the decoder target data for each sample
    decoder_target_data= []

    # Each entry of this list represents the FG value to be predicted for each example
    y = []

    buckects = {}

    pos = 0

    # For each patient
    for index, rows in dataset.groupby("Id"):

        if rows.shape == 1:
            continue

        # Select all columns except the column 'Id'
        rows = rows.iloc[:, 1:]

        num_visits = rows.shape[0]

        for visit in range(1, num_visits):

            visit_data = []

            rows.iloc[:visit, :].apply(
                lambda row: visit_data.extend(list(row.values)), axis=1)
            
            # El decoder recibira el ultimo FG conocido. Es decir el primer FG pasado al decoder,
            # es el ultimo valor de FG que se le pasa al encoder (el último FG de visita). El resto de valores de FG no son pasados al encoder.
            
            FG_values = rows["Filtrado_glomerular"].values[(visit-1):].tolist()

            encoder_input_data.append(visit_data)
            # El decoder_input_data tendra los valores de FG que se quieren predecir +1, el adicional es el ultimo 
            #valor conocido de FG
            decoder_input_data.append(FG_values[:-1])
            # El decoder_target_data tendra los valores de FG que se quieren predecir
            decoder_target_data.append(FG_values[1:])
            
            key = (visit, len(FG_values)-1)
            if key not in buckects:
                buckects[key] = [pos]
            else:
                buckects[key].append(pos)

            pos += 1
            
    #decoder_target_data = sequence.pad_sequences(decoder_target_data, dtype="float32", padding='post', value= -1000).tolist()

    return encoder_input_data, decoder_input_data, decoder_target_data, buckects

In [100]:
def data_generator(encoder_X, decoder_X, decoder_y, num_features):

    i = 0
    nsamples = len(encoder_X)

    while True:

        enc_x = encoder_X[i]  # get the list representing the predictive vars
        dec_x = decoder_X[i]
        dec_y = decoder_y[i]

        i += 1

        timesteps_encoder = int(len(enc_x) / num_features)
        timesteps_decoder = len(dec_x)

        enc_x = np.array(enc_x).reshape((1, timesteps_encoder, num_features))
        dec_x = np.array(dec_x).reshape((1, timesteps_decoder, 1))
        dec_y = np.array(dec_y).reshape((1, timesteps_decoder, 1))
        
        yield [enc_x, dec_x], dec_y

        # Reset the counter
        if i == nsamples:
            i = 0

In [101]:
class DataSequence(Sequence):

    def __init__(self, x_encoder, x_decoder, y_decoder, batch_size, num_features, buckets):

        self.batch_size = batch_size
        self.num_features = num_features
        self.X_encoder = []
        self.X_decoder = []
        self.y_decoder = []

        for key, list_indexes in buckets.items():

            permut = np.random.permutation(list_indexes)

            split_indices = [int(self.batch_size*(i+1))
                             for i in np.arange(np.floor(permut.size / self.batch_size))]

            list_arrays = np.array_split(permut, split_indices)

            for l in list_arrays:

                if len(l) == 0:
                    break

                x_encoder_array = []
                x_decoder_array = []
                y_decoder_array = []

                for i in l:
                    x_encoder_array.append(x_encoder[i])
                    x_decoder_array.append(x_decoder[i])
                    y_decoder_array.append(y_decoder[i])

                x_encoder_array = np.array(x_encoder_array).reshape(
                    (l.size, key[0], self.num_features))
                x_decoder_array = np.array(x_decoder_array).reshape(
                    (l.size, key[1], 1))
                y_decoder_array = np.array(y_decoder_array).reshape(
                    (l.size, key[1], 1))

                self.X_encoder.append(x_encoder_array)
                self.X_decoder.append(x_decoder_array)
                self.y_decoder.append(y_decoder_array)

    def __len__(self):
        return len(self.y_decoder)
    
    def size(self):
        return len(self.y_decoder)

    def __getitem__(self, idx):

        if self.X_encoder[idx].shape[0] == 0:
            import sys
            sys.exit("Alert: Batch of size 0")

        return [self.X_encoder[idx], self.X_decoder[idx]], self.y_decoder[idx]

    def subset(self, indexes):

        empty_data = DataSequence(
            [], [], [], self.batch_size, self.num_features, {})

        for index in indexes:

            [X_encoder_temp, X_decoder_temp], y_decoder_temp = self[index]

            empty_data.X_encoder.append(X_encoder_temp)
            empty_data.X_decoder.append(X_decoder_temp)
            empty_data.y_decoder.append(y_decoder_temp)

        return empty_data

In [4]:
maskValue= -1000

def custom_loss(yTrue,yPred):
    
    #find which values in yTrue (target) are the mask value
    isMask = K.equal(yTrue, maskValue) #true for all mask values

    #transform to float (0 or 1) and invert
    isMask = K.cast(isMask, dtype=K.floatx())
    isMask = 1 - isMask #now mask values are zero, and others are 1
    
    #multiply this by the inputs:
    #maybe you might need K.expand_dims(isMask) to add the extra dimension removed by K.all
    yTrue = yTrue * isMask  
    yPred = yPred * isMask
    
    def _logcosh(x):
        return x + nn.softplus(-2. * x) - math_ops.log(2.)

    return K.mean(_logcosh(yPred - yTrue), axis=-1)

In [102]:
imputation_method= "mean-average_last_next_values"

dataset = pd.read_csv(
        "datasets/datos-demograficos-visitas-tratamientos-missing-imputation-{method}-allnumerics.csv".format(
            method=imputation_method))

dataset_original = pd.read_csv(
        "datasets/datos-demograficos-visitas-tratamientos-missing-imputation-{method}.csv".format(
            method=imputation_method))

num_features = dataset.shape[1] - 1  # Id is not considered

scaler = StandardScaler()
scaler.fit(dataset_original["Filtrado_glomerular"].values.reshape(-1,1))

encoder_input_data, decoder_input_data, decoder_target_data, buckets = transform_dataset_different_timesteps(
        dataset)

In [54]:
nepochs = 10
nsamples= len(encoder_input_data)

In [105]:
# Define an input sequence and process it.
def create_model(print_summary = False, plot_model = False):
    latent_dim = 256

    encoder_inputs = Input(shape=(None, num_features), name="encoder_input")
    encoder = LSTM(latent_dim, return_state=True, name= "encoder_layer")
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard encoder_outputs and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using encoder_states as initial state.
    decoder_inputs = Input(shape=(None, 1), name= "decoder_input")
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the 
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,  name= "decoder_layer")
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states) 
   
    decoder_dense = Dense(1, name="dense_layer")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # encoder_input_data & decoder_input_data into decoder_target_data
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    if print_summary:
        print(model.summary())
        
    model.compile(loss="logcosh", optimizer='adam')
    
    if plot_model:
        plot_model(model, to_file='model.png', show_shapes=True)
        
    return model

def fit_model_generator(model):
    return model.fit_generator(generator(encoder_input_data, decoder_input_data, decoder_target_data, num_features), steps_per_epoch = nsamples, epochs = nepochs, use_multiprocessing= False)
    
def fit_model_sequence(model, dataTrain, dataTest, nepochs):
    return model.fit_generator(dataTrain, epochs=nepochs, verbose=1, validation_data= dataTest, use_multiprocessing=False)


In [159]:
# Example of prediction with the trained model
i = 15

enc_x = encoder_input_data[i]  # get the list representing the predictive vars
dec_x = decoder_input_data[i]
dec_y = decoder_target_data[i]

timesteps_encoder = int(len(enc_x) / num_features)
timesteps_decoder = len(dec_x)

enc_x = np.array(enc_x).reshape((1, timesteps_encoder, num_features))
dec_x = np.array(dec_x).reshape((1, timesteps_decoder, 1))
dec_y = np.array(dec_y).reshape((1, timesteps_decoder, 1))
        
prediction = model.predict([enc_x, dec_x])[0]

In [153]:
scaler.inverse_transform(prediction)

array([[63.0442  ],
       [63.429577]], dtype=float32)